In [1]:
import numpy as np
import torch
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset

In [2]:
class Sum_Dataset(Dataset):
    def __init__(self, data_path, lab_path):
        data = np.load(data_path)
        labels = np.load(lab_path)
        self.images = torch.tensor(data, dtype=torch.float32) / 255.0
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [3]:
class Standard_CNN_model(torch.nn.Module):
    def __init__(self):
        super(Standard_CNN_model, self).__init__()
        self.cnn = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2),
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2)
        )
        self.fc = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(64 * 10 * 42, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 1)
        )

    def forward(self, x):
        x = self.cnn(x)
        # print(x.shape)
        x = self.fc(x)
        return x


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

test_dataset = Sum_Dataset('../DL-Project/data2.npy', '../DL-Project/lab2.npy')
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

Using device: cuda


In [15]:
model = Standard_CNN_model().to(device)
model.load_state_dict(torch.load('digit_sum_model.pth'))

model.eval()
acc = 0

for images, labels in tqdm(test_dataloader):
    images, labels = images.unsqueeze(1).to(device), labels.to(device)
    outputs = model(images)
    predictions = outputs.round().flatten()
    acc += (predictions == labels).sum().item()


accuracy = acc / (len(test_dataloader) * test_dataloader.batch_size)
print(f"Accuracy: {accuracy * 100:.2f}%")

/tmp/ipykernel_14259/1316166101.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('digit_sum_model.pth'))
100%|██████████| 313/313 [00:00<

Accuracy: 10.55%
